# [Project Euler](https://ProjectEuler.net)
[This Python 3 notebook](Project%20Euler%20%28Python%203%29.ipynb) contains *some* solutions for the [Project Euler](https://ProjectEuler.net) challenge.

### /!\ **Warning:** do not spoil yourself the pleasure of solving these problems by yourself!

[I (Lilian Besson)](http://perso.crans.org/besson/) started to work again on Project Euler in October 2020
I should try to work on it again, hence this notebook...

![Badge giving the number of solved problems](https://ProjectEuler.net/profile/Naereen.png?h "Badge giving the number of solved problems")

---
## Common tool

Let's write here a few efficient functions that are used in lots of problems.

In [277]:
%load_ext Cython

The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython


In [278]:
%%cython
import math

def erathostene_sieve(int n):
    cdef list primes = [False, False] + [True] * (n - 1)  # from 0 to n included
    cdef int max_divisor = math.floor(math.sqrt(n))
    cdef int i = 2
    for divisor in range(2, max_divisor + 1):
        if primes[divisor]:
            number = 2*divisor
            while number <= n:
                primes[number] = False
                number += divisor
    return primes

In [ ]:
sieve10million = erathostene_sieve(int(1e7))
primes_upto_10million = [p for p,b in enumerate(sieve10million) if b]
print(f"There are {len(primes_upto_10million)} prime numbers smaller than 10 million")

---
## [Problem 51: prime  (pastis ! 51 je t'aime)](https://projecteuler.net/problem=51)

TODO

---
## [Problem 52: ](https://projecteuler.net/problem=52)

TODO

---
## [Problem 53: ](https://projecteuler.net/problem=53)

TODO

---
## [Problem 54: ](https://projecteuler.net/problem=54)

TODO

---
## [Problem 55: ](https://projecteuler.net/problem=55)

TODO

---
## Continue